## 상품 선별해서 전처리
- 날씨와 상관성이 낮은 상품들을 제외한 후, 데이터를 전처리하는 코드입니다.
- 날씨 변수의 경우, 평균기온/최저기온/최고기온/최소상대습도/평균상대습도/O3/PM10를 선택했습니다.

In [1]:
import pandas as pd
import datetime as dt
from sklearn.model_selection import train_test_split

In [2]:
sale_df = pd.read_csv('../data/sale_data.csv') # 원핫 인코딩이 되지 않고, 선호도 점수가 계산이 되지 않은 판매 데이터
sale_ohe = pd.read_csv('../data/sale_data_ohe.csv',encoding='CP949') # 원핫 인코딩이 되고, 선호도 점수가 계산된 판매 데이터
weather_df = pd.read_csv('../data/weather_data.csv', encoding = 'CP949') # 날씨 데이터
weather_df = weather_df[['일시','평균기온(°C)','최저기온(°C)','최고기온(°C)','최소 상대습도(%)','평균 상대습도(%)','O3','PM10']] # 해당 날씨변수만 추출합니다.

In [11]:
# 판매 데이터에 선호도 점수만 추가합니다.
sale_df = pd.concat([sale_df, sale_ohe.loc[:,'일별 물품 선호도 점수':'60대 여성 선호도 점수']], axis=1)

In [12]:
sale_df.head()

,날짜,대분류,소분류,20대 남성 판매량(개),20대 여성 판매량(개),30대 남성 판매량(개),30대 여성 판매량(개),40대 남성 판매량(개),40대 여성 판매량(개),50대 남성 판매량(개),...,20대 남성 선호도 점수,20대 여성 선호도 점수,30대 남성 선호도 점수,30대 여성 선호도 점수,40대 남성 선호도 점수,40대 여성 선호도 점수,50대 남성 선호도 점수,50대 여성 선호도 점수,60대 남성 선호도 점수,60대 여성 선호도 점수
0,2018-01-01,식품,가공란,13.0,37.0,6.0,16.0,6.0,9.0,0.0,...,0.342493,0.329235,0.073506,0.079250,0.078651,0.054480,0.000000,0.042386,0.000000,0.0
1,2018-01-01,냉난방가전,가열식 가습기,1.0,1.0,0.0,0.0,2.0,3.0,0.0,...,0.330888,0.111758,0.000000,0.000000,0.329272,0.228082,0.000000,0.000000,0.000000,0.0
2,2018-01-01,식품,가자미,0.0,3.0,2.0,5.0,1.0,4.0,3.0,...,0.000000,0.087430,0.080248,0.081112,0.042933,0.079304,0.263728,0.092548,0.272696,0.0
3,2018-01-01,식품,갈비/찜/바비큐용 돈육,0.0,7.0,7.0,4.0,1.0,2.0,0.0,...,0.000000,0.300616,0.413882,0.095620,0.063265,0.058430,0.000000,0.068188,0.000000,0.0
4,2018-01-01,식품,갈비용 우육,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.000000,0.595136,0.000000,0.000000,0.000000,0.404864,0.000000,0.000000,0.000000,0.0


In [13]:
# 날씨와 상관성이 낮아 제외할 상품 목록입니다.
condition = ['가공란',
 '옥돔',
 '오징어',
 '오메가3/스쿠알렌 영양제',
 '오리고기/훈제오리',
 '영지버섯',
 '연어/훈제연어',
 '옥수수차',
 '어린잎/새싹채소',
 '어린이홍삼',
 '어린이영양제',
 '어란(생선알)',
 '양파/마늘즙',
 '양배추/양상추',
 '장조림/카레용 돈육',
 '잡곡 씨드류',
 '잡곡',
 '인스턴트커피',
 '인삼/수삼/산삼',
 '윙봉/닭다리/날개',
 '원두커피',
 '우엉/연근',
 '요거트/발효유',
 '옻/칡/쑥즙',
 '양념우육',
 '양념 돈육',
 '야채/호박즙',
 '소고기 등심/안심',
 '선식',
 '생식/선식 분말',
 '생선류',
 '생닭/닭부분육',
 '소고기 육회',
 '저/무지방우유',
 '수입우육',
 '아몬드유/코코넛밀크',
 '쌈채소',
 '쌀',
 '스피루리나 영양제',
 '숙취/에너지/건강 음료',
 '수산 생물',
 '전복 생물',
 '전통주',
 '헛개/가시오가피',
 '해조류 미역',
 '해조류 다시마',
 '한우육',
 '한방재료',
 '한방 분말/환제품',
 '프로폴리스/로얄젤리',
 '프로바이오틱스',
 '프라페/버블티 파우더',
 '흰우유',
 '환자식',
 '홍어',
 '홍삼절편/홍삼정과',
 '홍삼액/홍삼정',
 '홍삼/인삼 제품',
 '홍삼 음료',
 '홍삼 분말/환',
 '홍삼 간식',
 '혼합견과',
 '호박',
 '커피용 프림',
 '캡슐/POD커피',
 '칼슘/철분 영양제',
 '카페용 초콜릿시럽',
 '초코우유',
 '초유 영양제',
 '차/곡물 음료',
 '차 선물세트',
 '주꾸미',
 '젓갈류',
 '커피음료',
 '콜라겐/코큐텐 영양제',
 '표고버섯',
 '포도/거봉/체리',
 '파김치',
 '파/양파',
 '클로렐라 영양제',
 '콩나물/숙주',
 '기타 한방/환제품',
 '기타 농산물',
 '글루코사민/키토산 영양제',
 '굴비/조기',
 '국내산 돈육',
 '과일세트',
 '꽃게',
 '낙지',
 '다이어트보조식',
 '느타리버섯',
 '녹차',
 '나물',
 '곶감/반건시',
 '건어물 노가리',
 '건어물 건새우',
 '건바나나',
 '건망고',
 '건강즙/녹용',
 '건강즙',
 '갓김치',
 '감마리놀렌산 영양제',
 '갈치',
 '갈비용 우육',
 '갈비/찜/바비큐용 돈육',
 '가자미',
 '건어물 마른오징어',
 '과실차',
 '건어물 멸치',
 '건어물 진미채',
 '고등어',
 '견과류 호두',
 '견과류 피스타치오',
 '견과류 캐슈넛',
 '견과류 카카오닙스',
 '견과류 잣/은행',
 '견과류 마카다미아',
 '견과류 땅콩',
 '견과류',
 '게장류',
 '건자두',
 '건어물 황태',
 '건어물 쥐포',
 '닭 양념육',
 '과채 음료/주스',
 '닭가슴살',
 '보리차',
 '백김치',
 '부추',
 '반찬류',
 '새우/대하',
 '새송이버섯',
 '상황버섯',
 '삼치',
 '비타민/화이바 음료',
 '비타민',
 '브로콜리/셀러리',
 '반건조고구마',
 '마늘/생강',
 '루테인/눈 영양제',
 '랍스타',
 '딸기우유',
 '두유',
 '돼지 곱창',
 '도라지/더덕',
 '메추리알',
 '명태/동태',
 '무/배추',
 '바나나우유',
 '바나나/파인애플/망고',
 '믹스 채소',
 '미나리',
 '문어',
 '밀크티/티라떼',
 '혼합곡',
 '구이/수육용 돈육',
 '세안도구',
 '샴푸',
 '색조 메이크업 아이브로우',
 '색조 메이크업 아이라이너',
 '색조 메이크업 쉐딩/하이라이터',
 '색조 메이크업 속눈썹영양제',
 '손톱정리도구',
 '스킨케어 필오프팩',
 '스킨케어 워시오프팩',
 '스킨케어 시트마스크팩',
 '스킨케어 수면팩',
 '스킨케어 마스크팩',
 '스킨케어 곡물팩',
 '스크럽/필링크림',
 '헤어스타일링용 펌제',
 '헤어스타일링용 염색약',
 '헤어무스',
 '헤어매니큐어',
 '헤어 브러쉬',
 '핸드워시/스크럽',
 '헤어스프레이',
 '풋패치',
 '헤어에센스',
 '헤어젤',
 '화장 퍼프',
 '화장 비누',
 '헤어케어세트',
 '헤어왁스',
 '색조 메이크업 볼터치',
 '풋스프레이',
 '풋워시/스크럽',
 '팩도구',
 '트리트먼트',
 '태닝용 선크림',
 '클렌징 폼',
 '클렌징 티슈',
 '클렌징 크림',
 '클렌징 워터/젤',
 '클렌징 오일',
 '클렌징 로션',
 '기초 화장용 오일/앰플',
 '기초 화장용 에센스',
 '기초 화장용 스킨',
 '기초 화장용 미스트',
 '기능성 화장품 세트',
 '기능성 트러블케어 화장품',
 '기능성 영양보습 화장품',
 '기능성 모공관리 화장품',
 '기능성 링클케어 화장품',
 '네일케어',
 '네일컬러',
 '네일세트',
 '네일리무버',
 '네일관리 소품',
 '남성 클렌징',
 '남성 크림',
 '남성 에센스',
 '남성 스킨',
 '남성 쉐이빙',
 '남성 메이크업',
 '색조 메이크업 립틴트',
 '베이스 메이크업용 프라이머',
 '베이스 메이크업용 파운데이션',
 '베이스 메이크업용 트윈케이크',
 '베이스 메이크업용 컨실러',
 '베이스 메이크업용 메이크업베이스',
 '베이스 메이크업용 가루파우더',
 '베이스 메이크업용 CC크림',
 '베이스 메이크업용 BB크림',
 '베이스 메이크업 세트',
 '뷰티 눈썹정리도구',
 '뷰티 손거울',
 '색조 메이크업 립라이너',
 '뷰티용 여드름압출기',
 '뷰티용 샤프너',
 '뷰티용 뷰러',
 '뷰티용 면봉/귀이개',
 '뷰티용 기름종이',
 '뷰티 화장품 공병/케이스',
 '뷰티 화장솜',
 '뷰티 헤어캡',
 '뷰티 속눈썹/쌍꺼풀',
 '메이크업 브러쉬',
 '립앤아이 리무버',
 '린스',
 '바디케어용 청 결제',
 '메이크업 박스',
 '바디 세트',
 '바디케어용 슬리밍',
 '바디케어용 볼륨업크림',
 '바디케어용 때비누',
 '바디 스크럽',
 '미용가위',
 '네일아트',
 '온열매트',
 '의류건조기',
 '신발건조기',
 '보일러',
 '벽걸이형 냉온풍기']

In [14]:
# 판매 데이터의 행 인덱스를 상품 소분류로 변환합니다.
sale_df.index = sale_df.소분류

# 앞서 정의한 condition에 해당하는 행을 삭제합니다.
sale_df.drop(condition, axis=0, inplace=True)

In [15]:
# 상품 소분류로 원 핫 인코딩 진행
sale_df = pd.get_dummies(data = sale_df, columns = ['소분류'])

In [16]:
# 행 인덱스를 숫자로 초기화
sale_df.index = range(0, len(sale_df))

In [17]:
# 병합하려고 했는데 날짜 column name이 달라서 '날짜'로 통일
weather_df.rename(columns = {'일시':'날짜'}, inplace = True)

In [18]:
# pandas의 merge_asof method 사용하여 '날짜'를 기준으로 합치는 방법 사용하려고 했는데, '날짜'의 type이 str이라서 integer나 float으로 변경 필요
# pandas의 to_datetime method를 사용하여 type 변경
sale_df['날짜'] = pd.to_datetime(sale_df['날짜'], format='%Y-%m-%d')
sale_df = sale_df.sort_values('날짜')
weather_df['날짜'] = pd.to_datetime(weather_df['날짜'], format = '%Y-%m-%d')
weather_df = weather_df.sort_values('날짜')

In [19]:
# df 순서를 바꾸면 안됨
sale_and_weather_df = pd.merge_asof(sale_df, weather_df, on='날짜', allow_exact_matches=True)

In [20]:
sale_and_weather_df

,날짜,대분류,20대 남성 판매량(개),20대 여성 판매량(개),30대 남성 판매량(개),30대 여성 판매량(개),40대 남성 판매량(개),40대 여성 판매량(개),50대 남성 판매량(개),50대 여성 판매량(개),...,소분류_회,소분류_휴대용 선풍기,소분류_히터,평균기온(°C),최저기온(°C),최고기온(°C),최소 상대습도(%),평균 상대습도(%),O3,PM10
0,2018-01-01,냉난방가전,1.0,1.0,0.0,0.0,2.0,3.0,0.0,0.0,...,0,0,0,0.066157,-3.463993,4.734708,20.763179,41.267581,0.015544,44.032069
1,2018-01-01,식품,7.0,12.0,6.0,18.0,6.0,8.0,3.0,2.0,...,0,0,0,0.066157,-3.463993,4.734708,20.763179,41.267581,0.015544,44.032069
2,2018-01-01,식품,1.0,9.0,3.0,7.0,3.0,10.0,2.0,4.0,...,0,0,0,0.066157,-3.463993,4.734708,20.763179,41.267581,0.015544,44.032069
3,2018-01-01,식품,2.0,4.0,5.0,1.0,3.0,6.0,3.0,5.0,...,0,0,0,0.066157,-3.463993,4.734708,20.763179,41.267581,0.015544,44.032069
4,2018-01-01,식품,52.0,225.0,200.0,459.0,107.0,219.0,34.0,52.0,...,0,0,0,0.066157,-3.463993,4.734708,20.763179,41.267581,0.015544,44.032069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80379,2019-12-31,냉난방가전,0.0,0.0,3.0,1.0,2.0,0.0,1.0,0.0,...,0,0,0,-5.197603,-8.010086,-1.050035,26.748097,40.298909,0.023901,22.734047
80380,2019-12-31,뷰티,3.0,3.0,5.0,7.0,4.0,3.0,2.0,2.0,...,0,0,0,-5.197603,-8.010086,-1.050035,26.748097,40.298909,0.023901,22.734047
80381,2019-12-31,식품,2.0,14.0,2.0,7.0,1.0,8.0,1.0,8.0,...,0,0,0,-5.197603,-8.010086,-1.050035,26.748097,40.298909,0.023901,22.734047
80382,2019-12-31,뷰티,21.0,64.0,42.0,95.0,52.0,77.0,16.0,30.0,...,0,0,0,-5.197603,-8.010086,-1.050035,26.748097,40.298909,0.023901,22.734047


In [21]:
# 대분류 열은 삭제합니다.
sale_and_weather_df.drop(['대분류'], axis = 1, inplace = True)

In [22]:
# 일별 판매량 합계 데이터에 오류가 있어 재정의하겠습니다.
sale_and_weather_df.합계 = sale_and_weather_df.loc[:,'20대 남성 판매량(개)': '60대 여성 판매량(개)'].sum(axis=1)

In [24]:
# 상품 소분류 + 날씨 변수가 input data입니다.
X_df = pd.concat([sale_and_weather_df.날짜, sale_and_weather_df.loc[:, '소분류_가스온수기':]],axis=1)
X_df

,날짜,소분류_가스온수기,소분류_가열식 가습기,소분류_감/홍시,소분류_감귤/한라봉/오렌지,소분류_감말랭이,소분류_감자,소분류_건대추,소분류_건포도,소분류_견과류 밤,...,소분류_회,소분류_휴대용 선풍기,소분류_히터,평균기온(°C),최저기온(°C),최고기온(°C),최소 상대습도(%),평균 상대습도(%),O3,PM10
0,2018-01-01,0,1,0,0,0,0,0,0,0,...,0,0,0,0.066157,-3.463993,4.734708,20.763179,41.267581,0.015544,44.032069
1,2018-01-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0.066157,-3.463993,4.734708,20.763179,41.267581,0.015544,44.032069
2,2018-01-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0.066157,-3.463993,4.734708,20.763179,41.267581,0.015544,44.032069
3,2018-01-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0.066157,-3.463993,4.734708,20.763179,41.267581,0.015544,44.032069
4,2018-01-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0.066157,-3.463993,4.734708,20.763179,41.267581,0.015544,44.032069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80379,2019-12-31,0,0,0,0,0,0,0,0,0,...,0,0,0,-5.197603,-8.010086,-1.050035,26.748097,40.298909,0.023901,22.734047
80380,2019-12-31,0,0,0,0,0,0,0,0,0,...,0,0,0,-5.197603,-8.010086,-1.050035,26.748097,40.298909,0.023901,22.734047
80381,2019-12-31,0,0,0,0,0,0,0,0,0,...,0,0,0,-5.197603,-8.010086,-1.050035,26.748097,40.298909,0.023901,22.734047
80382,2019-12-31,0,0,0,0,0,0,0,0,0,...,0,0,0,-5.197603,-8.010086,-1.050035,26.748097,40.298909,0.023901,22.734047


In [25]:
# 판매량과 선호도 잠수가 output data입니다.
y_df = sale_and_weather_df.loc[:, '20대 남성 판매량(개)':'60대 여성 선호도 점수']
y_df

,20대 남성 판매량(개),20대 여성 판매량(개),30대 남성 판매량(개),30대 여성 판매량(개),40대 남성 판매량(개),40대 여성 판매량(개),50대 남성 판매량(개),50대 여성 판매량(개),60대 남성 판매량(개),60대 여성 판매량(개),...,20대 남성 선호도 점수,20대 여성 선호도 점수,30대 남성 선호도 점수,30대 여성 선호도 점수,40대 남성 선호도 점수,40대 여성 선호도 점수,50대 남성 선호도 점수,50대 여성 선호도 점수,60대 남성 선호도 점수,60대 여성 선호도 점수
0,1.0,1.0,0.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,...,0.330888,0.111758,0.000000,0.000000,0.329272,0.228082,0.000000,0.000000,0.000000,0.000000
1,7.0,12.0,6.0,18.0,6.0,8.0,3.0,2.0,0.0,1.0,...,0.245554,0.142176,0.097873,0.118711,0.104723,0.064480,0.107216,0.037624,0.000000,0.081642
2,1.0,9.0,3.0,7.0,3.0,10.0,2.0,4.0,0.0,0.0,...,0.067917,0.206451,0.094746,0.089381,0.101378,0.156051,0.138388,0.145689,0.000000,0.000000
3,2.0,4.0,5.0,1.0,3.0,6.0,3.0,5.0,0.0,1.0,...,0.119044,0.080414,0.138391,0.011190,0.088847,0.082058,0.181924,0.159602,0.000000,0.138530
4,52.0,225.0,200.0,459.0,107.0,219.0,34.0,52.0,5.0,9.0,...,0.101936,0.148972,0.182313,0.169164,0.104365,0.098642,0.067904,0.054666,0.030976,0.041062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80379,0.0,0.0,3.0,1.0,2.0,0.0,1.0,0.0,1.0,0.0,...,0.000000,0.000000,0.206447,0.027823,0.147265,0.000000,0.150771,0.000000,0.467694,0.000000
80380,3.0,3.0,5.0,7.0,4.0,3.0,2.0,2.0,1.0,0.0,...,0.180675,0.061023,0.140026,0.079258,0.119862,0.041513,0.122715,0.064595,0.190332,0.000000
80381,2.0,14.0,2.0,7.0,1.0,8.0,1.0,8.0,0.0,0.0,...,0.120342,0.284519,0.055960,0.079187,0.029939,0.110603,0.061302,0.258147,0.000000,0.000000
80382,21.0,64.0,42.0,95.0,52.0,77.0,16.0,30.0,1.0,7.0,...,0.119717,0.123229,0.111339,0.101820,0.147498,0.100860,0.092928,0.091717,0.018017,0.092876


In [108]:
# input 및 output data를 무작위로 섞어 train, test data를 생성합니다.
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size = 0.3, shuffle = True, random_state = 1004)

In [26]:
# RNN의 경우, 데이터를 무작위로 섞으면 되지 않기 때문에 input 및 output data의 원본을 Rnn용 데이터로 저장합니다.
X_df.to_csv('../data/X_Rnn.csv', index = False, encoding='CP949')
y_df.to_csv('../data/y_Rnn.csv', index = False, encoding='CP949')

In [109]:
# 각 데이터를 저장합니다.
X_train.to_csv('../data/X_train_weather_columns_6.csv', index = False, encoding='CP949')
X_test.to_csv('../data/X_test_weather_columns_6.csv', index = False, encoding='CP949')
y_train.to_csv('../data/y_train_weather_columns_6.csv', index = False, encoding='CP949')
y_test.to_csv('../data/y_test_weather_columns_6.csv', index = False, encoding='CP949')